In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
import numpy as np
from tensorflow.keras.preprocessing import image
import os


In [3]:
DATASET_DIR = "animals"   # dataset folder
ANIMALS_TXT = "name of the animals.txt"  # text file with 90 animal names
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123


In [4]:
with open(ANIMALS_TXT, "r") as f:
    animal_names = [line.strip() for line in f.readlines()]

print("✅ Loaded", len(animal_names), "animal classes")


✅ Loaded 90 animal classes


In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,   # 80% train, 20% validation
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 5400 files belonging to 90 classes.
Using 4320 files for training.
Found 5400 files belonging to 90 classes.
Using 1080 files for validation.


In [7]:
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [9]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # freeze base model

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(len(animal_names), activation="softmax")  # 90 classes
])
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ------------------------
# 5. Training
# ------------------------
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 90)                2

In [10]:
model.save("animal90_classifier.h5")
print("✅ Model saved as animal90_classifier.h5")

c:\Users\Kavya KS\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


✅ Model saved as animal90_classifier.h5


In [11]:
def predict_animal(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_animal = animal_names[predicted_index]
    confidence = prediction[0][predicted_index]

    return predicted_animal, confidence

In [14]:
#test_img = os.path.join(DATASET_DIR, animal_names[0], os.listdir(os.path.join(DATASET_DIR, animal_names[0]))[0])
test_img="lion.png"
predicted_animal, conf = predict_animal(test_img)
print(f"🔮 Predicted: {predicted_animal} (Confidence: {conf:.2f})")


1/1 [==============================] - 0s 86ms/step
🔮 Predicted: lion (Confidence: 0.90)
